In [ ]:
from matplotlib import animation, rc, pyplot as plt
import numpy as np
from numpy import genfromtxt
from scipy.optimize import curve_fit

In [ ]:
wbnlist = genfromtxt('Corrected Potential of 2D Hydrogen.csv', delimiter=',',dtype=complex)

In [ ]:
# ------------------------------------------------------------
# Configuration:
# In `__init__`, select either the corrected or uncorrected potential.
# In `wave_packet`, select either the corrected or uncorrected wavefunction.
# Uncomment the desired option in each section.
# ------------------------------------------------------------

class GroundHydrogen:
    def __init__(self,
                 mass = 1, hbar=1,
                 xmin=-5, xmax=5, ymin=-5, ymax=5, ninterval=128):
        
        self.xx = np.linspace(xmin, xmax, ninterval)   # discretise the space
        self.dx = self.xx[1] - self.xx[0]
        self.yy = np.linspace(ymin, ymax, ninterval)   
        self.dy = self.yy[1] - self.yy[0]
        self.x, self.y = np.meshgrid(self.xx, self.yy)
        
        self.hbar = hbar
        self.mass = mass
        self.Z = 1
        self.r = np.sqrt(self.x**2 + self.y**2)
        
        res = ninterval 
        self.dkx = 2 * np.pi / (res * self.dx)
        self.kx = np.concatenate((np.arange(0, res / 2),
                                 np.arange(-res / 2, 0))) * self.dkx
        self.dky = 2 * np.pi / (res * self.dy)
        self.ky = np.concatenate((np.arange(0, res / 2),
                                 np.arange(-res / 2, 0))) * self.dky
        self.KX, self.KY = np.meshgrid(self.kx, self.ky)

        self.Vhat = np.ones((len(self.x),len(self.y)),dtype=complex)   
        #self.Vhat *= -1/np.sqrt(self.x**2+self.y**2)  #This is the uncorrected potential.
        self.Vhat *= wbnlist  #This is the corrected potential.
        
        self.Khat = np.ones((len(self.x),len(self.y)),dtype=complex)
        self.Khat *= (self.KX**2 + self.KY**2) /(2*mass) 
        
        self.wave = self.wave_packet(self.x, self.y)

        self.history = {}

    
    def evolve(self, tfinal=6, nt=6000):  
        t = np.linspace(0, tfinal, nt)
        dt = t[1] - t[0]

        Vt = np.exp(-1j * (1/2) * self.Vhat * dt / self.hbar)
        Kt = np.exp(-1j * (1/1) * self.Khat * dt / self.hbar)

        t_list=[]
        auto_list=[]        
        
        psi = np.copy(self.wave)
        t_list.append(t[0])
        auto_list.append(np.sum(np.conjugate(self.wave) * psi * self.dx* self.dy))
               
        for i in range(nt):
            
            psi *= Vt 

            psi = np.fft.fft2(psi) 
            psi *= Kt
            psi = np.fft.ifft2(psi)

            psi *= Vt 

            t_list.append(t[i])
            auto_list.append(np.sum(np.conjugate(self.wave) * psi * self.dx * self.dy))
                       
        return t_list, auto_list
                    
    def wave_packet(self, x, y, Edynamic=-1.757):

        psi = np.ones((len(self.x),len(self.y)),dtype=complex)
                
        #Parameters needed for wavefunction correction:
        #Here self.E (and self.alpha) must be uniquely determined for each specific grid density:
        self.E = Edynamic    #This is the E_dynamic used for wavefunction correction 
        self.alpha = np.sqrt(-2*self.E)    #This is the alpha used for wavefunction correction
        self.gamma = self.Z/self.alpha - 1/2 

        
        #psi *= np.exp(-2*self.r)   # This is the uncorrected wavefunction.        
        psi *= self.r**self.gamma *np.exp(-self.alpha*self.r)   # This is the corrected wavefunction.
                
        initnorm = np.sum(np.conjugate(psi)*psi*self.dx*self.dy)
        psi *= 1/np.sqrt(initnorm)

        return psi

In [ ]:
qt = GroundHydrogen()

In [ ]:
t_list, auto_list = qt.evolve()

In [ ]:
#save the data
np.savetxt('hydrogen t_list.csv',t_list, delimiter=',')
np.savetxt('hydrogen autocorrelation.csv',auto_list, delimiter=',')

In [ ]:
#reload the data
t_list = genfromtxt('hydrogen t_list.csv', delimiter=',')
auto_list = genfromtxt('hydrogen autocorrelation.csv', delimiter=',', dtype=np.complex128)

In [ ]:
#plot the real part of autocorrelation
t = np.array(t_list)         
y_target = np.real(auto_list)     

def model(t, E):
    return np.cos(E * t)           

E0 = -1.75

popt, pcov = curve_fit(model, t, y_target, p0=[E0])
E_fit = popt[0]
E_err = np.sqrt(np.diag(pcov))[0] if pcov is not None else np.nan

print(fr"Fitted E_dynamic = {E_fit:.3f}")

plt.plot(t, y_target, label="Re{A(t)}", lw=1)
plt.plot(t, model(t, E_fit), lw=2,
         label=fr"$\cos({E_fit:.3f}\,t)$")
plt.legend()

plt.ylabel('Real Part of Autocorrelation')
plt.xlabel('Time (a.u.)')
plt.ylim(-1.03, 1.03)
plt.show()

In [ ]:
#plot the absolute autocorrelation
plt.plot(t_list, np.abs(auto_list))
plt.ylabel('Absolute Autocorrelation')
plt.xlabel('Time (a.u.)')
plt.ylim(0.979, 1.002)